In [3]:
import pandas
import torch
import torch.nn as nn
import torch.nn.functional as Fu
import numpy as np
from torch.autograd import Variable

In [4]:
data=np.loadtxt(open("extended.csv", "rb"), delimiter=",", skiprows=0, dtype=float)
print(data.shape)

(4368, 19)


In [5]:
real_data = torch.Tensor()
real_data = Variable(torch.from_numpy(data))
print(real_data)

Variable containing:
 0.0800  0.0800  0.2800  ...  -0.2000 -0.0400  0.0800
 0.1600  0.1600  0.0800  ...  -0.2000 -0.0400  0.0800
 0.4000  0.4000  0.4000  ...  -0.2000 -0.0400  0.0800
          ...             ⋱             ...          
 0.9600  0.9600  0.8400  ...   0.3600  0.5200  0.6400
-1.0000 -1.0000  0.8800  ...   0.4000  0.5600  0.6800
-0.9600 -0.9600  0.9200  ...   0.4400  0.6000  0.7200
[torch.DoubleTensor of size 4368x19]



In [9]:
import numpy.random
#returns 50 inputs for the Generator (3 features chord and 1 noise)
def generator_input(n):
    ret = Variable(torch.Tensor(n,4))
    tune = torch.rand(n)*2 -1
    tune2 = tune + 4/25
    tune3 = tune + 7/25
    ret[:,0]=tune
    ret[:,1]=tune2
    ret[:,2]=tune3
    ret[:,3]=torch.rand(n)*2 -1
    return ret

In [21]:
class Discri(nn.Module):
    def __init__(self):
        super(Discri, self).__init__()
        self.lin1 = nn.Linear(19, 19)
        self.lin2 = nn.Linear(19, 19)
        self.lin3 = nn.Linear(19, 1)
 
    def forward(self, x):
        x = Fu.leaky_relu(self.lin1(x))
        x = Fu.leaky_relu(self.lin2(x))
        return Fu.sigmoid(self.lin3(x))

In [62]:

class Gene(nn.Module):
    def __init__(self):
        super(Gene, self).__init__()
        self.lin1 = nn.Linear(4, 4)
        self.lin1.weights = torch.ones(4,4)
        self.lin2 = nn.Linear(4,16)
        self.lin2.weights = torch.ones(4,16)
        self.lin3 = nn.Linear(16, 16)
        self.lin3.weights = torch.ones(16,16)
        
    def forward(self, x):
        x = Fu.leaky_relu(self.lin1(x))
        x = Fu.dropout(Fu.leaky_relu(self.lin2(x)))
        return Fu.leaky_relu(self.lin3(x))

In [24]:
#generates a set of 4000 samples of data generated by the GENERATOR with random chords
def gene_fake(n):

    ret = Variable(torch.Tensor(n,19))
    gene_input = generator_input(n)
    ret[:,:16] = gene(gene_input).detach()
    ret[:,16:]=gene_input[:,:3]

    return ret

In [66]:
#initializations 

discri = Discri()
gene = Gene()

learning_rate = 1e-4
loss_disc = torch.nn.MSELoss()
loss_gene = torch.nn.MSELoss()
disc_optim = torch.optim.SGD(discri.parameters(), lr=learning_rate)
gene_optim = torch.optim.Adam(gene.parameters(), lr=learning_rate)

input = Variable(torch.Tensor(8368,19))
input[:4368] = real_data
input[4368:]= gene_fake(4000)

exp_output = Variable(torch.Tensor(8368))
exp_output[:4368] = torch.rand(4368)*0.4 + 0.8 #for real data
exp_output[4368:] = torch.rand(4000)*0.3 #dor fake data

batch_size = 50

In [67]:
for epoch in range(0,20):
    #training of discriminator
    for i in range(0,500):
        batch_index = (batch_size*i)%len(input)
        decision = discri(input[batch_index : (batch_index + batch_size)])
        loss = loss_disc(decision, exp_output[batch_index : (batch_index + batch_size)])      
        disc_optim.zero_grad()
        loss.backward(retain_graph=True)
        disc_optim.step()

    print("epoch :",epoch)
    print("discrimator loss :",loss.data[0])

    #training og generator
    for i in range(0,500):
        generated = gene_fake(2000)
        result = discri(generated)
        loss = loss_gene(result,Variable(torch.ones(2000)))
        gene_optim.zero_grad()
        loss.backward(retain_graph=True)
        gene_optim.step()
    print("generator loss :",loss.data[0])


epoch : 0
discrimator loss : 0.1268032193183899
generator loss : 0.2702048420906067
epoch : 1
discrimator loss : 0.12753024697303772
generator loss : 0.2691439986228943
epoch : 2
discrimator loss : 0.12825363874435425
generator loss : 0.2680729031562805
epoch : 3
discrimator loss : 0.12897302210330963
generator loss : 0.2670961022377014
epoch : 4
discrimator loss : 0.12968841195106506
generator loss : 0.2661060690879822
epoch : 5
discrimator loss : 0.1303998827934265
generator loss : 0.26487645506858826
epoch : 6
discrimator loss : 0.1311076134443283
generator loss : 0.2639181911945343
epoch : 7
discrimator loss : 0.13181152939796448
generator loss : 0.26294538378715515
epoch : 8
discrimator loss : 0.13251131772994995
generator loss : 0.2618851065635681
epoch : 9
discrimator loss : 0.1332070380449295
generator loss : 0.26078322529792786
epoch : 10
discrimator loss : 0.13389892876148224
generator loss : 0.2599372863769531
epoch : 11
discrimator loss : 0.13458691537380219
generator loss 

In [38]:
list(gene.lin1.parameters())

[Parameter containing:
 -0.0774 -0.2617 -0.4840 -0.3261
  0.0835  0.1075  0.4327 -0.0207
 -0.0357 -0.4261  0.4755 -0.0090
 -0.0083  0.2359  0.2078  0.1437
 [torch.FloatTensor of size 4x4], Parameter containing:
 -0.3659
  0.0495
 -0.0354
  0.4698
 [torch.FloatTensor of size 4]]

In [84]:
#generate blues song data CCCC FFCC GGCC
C = 60.0
G = 67.0
F = 65.0
ret = Variable(torch.Tensor(12,4))
tune = torch.from_numpy(np.array([C,C,C,C,F,F,C,C,G,G,C,C]))
tune2 = tune + 4
tune3 = tune + 7
ret[:,0]=tune
ret[:,1]=tune2
ret[:,2]=tune3
ret[:,3]=torch.randperm(12)
data = gene(ret).round()
print(data)

Variable containing:

Columns 0 to 12 
   -4     1    -4    -1     9    -7    -7    -4    -7     3     1    -5     1
   -5     1    -3    -1     9    -7    -8    -4    -6     4     2    -6     2
   -4     1    -4    -1     9    -7    -7    -4    -7     4     1    -6     1
   -4     1    -4    -1     9    -7    -7    -4    -7     4     1    -6     1
   -5     1    -3    -1    10    -8    -8    -4    -7     4     2    -7     2
   -5     1    -5    -2    10    -8    -8    -4    -8     4     1    -6     1
   -4     1    -4    -1     9    -7    -8    -4    -7     4     2    -6     1
   -5     1    -3    -1     9    -7    -8    -4    -6     4     2    -6     1
   -5     1    -5    -1    10    -8    -8    -4    -8     4     1    -6     1
   -5     1    -4    -1    10    -8    -9    -4    -7     4     2    -7     1
   -4     0    -5    -2     9    -7    -7    -3    -7     3     1    -5     0
   -4     1    -4    -1     9    -7    -8    -4    -6     4     2    -6     1

Columns 13 to 15 
   -6 

In [98]:
#convert to midi
from midiutil.MidiFile import MIDIFile

mf = MIDIFile(2)    

# the melody track
track = 0   
time = 0
mf.addTrackName(track, time, "Melody")
mf.addTempo(track, time, 120)
channel = 0
volume = 100

for bar in data:
    for note in bar:
        if note.data[0] >0:
            mf.addNote(track, channel, note.data[0]+40, time, 0.25, volume)
        time = time +0.25

#the chord track
track = 1
time = 0
volume = 50
mf.addTrackName(track,time,"Chords")    
mf.addTempo(track, time, 120)

C = 60
E = 64
G = 67
F = 65
A = 69
B = 71

for i in range(0,4):
    mf.addNote(track,channel,C,time,4,volume)
    mf.addNote(track,channel,E,time,4,volume)
    mf.addNote(track,channel,G,time,4,volume)
    time=time+4
for i in range(0,2):
    mf.addNote(track,channel,F,time,4,volume)
    mf.addNote(track,channel,A,time,4,volume)
    mf.addNote(track,channel,C,time,4,volume)
    time=time+4
for i in range(0,2): 
    mf.addNote(track,channel,C,time,4,volume)
    mf.addNote(track,channel,E,time,4,volume)
    mf.addNote(track,channel,G,time,4,volume)  
    time=time+4
for i in range(0,2): 
    mf.addNote(track,channel,G,time,4,volume)
    mf.addNote(track,channel,B,time,4,volume)
    mf.addNote(track,channel,E,time,4,volume)
    time=time+4
for i in range(0,2): 
    mf.addNote(track,channel,C,time,4,volume)
    mf.addNote(track,channel,E,time,4,volume)
    mf.addNote(track,channel,G,time,4,volume)  
    time=time+4

with open("awesome.mid", 'wb') as outf:
    mf.writeFile(outf)

/home/lamipaul/anaconda3/lib/python3.6/site-packages/midiutil/MidiFile.py:899: FutureWarning: Please explicitly set adjust_origin. Default behaviour will change in a future version.
  FutureWarning)


error: required argument is not an integer